In [ ]:
1. Reading the Dataset: The initial step involves reading the dataset from the CSV file using the pandas read_csv function to identify areas that require cleaning, recognizing patterns, and revealing potential issues such as missing or misjudged data, laying the foundation for subsequent cleaning processes. .
2. Constructing Functions to Cleaning Text Data
2.1. Handling Data Types
A function (transform_text) is applied to check cell value type. If a string is detected, the original value is retained; otherwise, it is converted to a string. Most records were affected by this process.
2.2. remove pattern contain Bracket and Quotation
A function (remove_pattern) is used to remove unwanted characters from text cells that start with "['" and end with "']".
2.3. Extracting Text from Raw Data
A function (extract_txtraw) is employed to extract text from a pattern that starts with digits and space or symbols.
2.4. Lowercasing Text Columns
Text columns ('question', 'model_answer', 'student_answer') are converted to lowercase.
2.5. Removing Symbolic Values
Functio (remove_symbolic_values) is defined to remove rows where text columns consist of only symbols.
3. Applying Transformations
The functions are applied to specific columns based on conditions using pandas apply and applymap functions.
4. Tracking Changes
The original dataframe is copied, and changes are tracked by comparing the original and modified dataframes.
5. Saving the Cleaned Dataset
The final cleaned dataset is saved to a new CSV file.
The data cleaning process ensures that the dataset is consistent and ready for subsequent analysis.

In [ ]:
import pandas as pd
import re

df = pd.read_csv(r"E:\Y4\Grad Proj\AGS\final_version.csv")

def transform_text(cell_value):
    # Check if the cell_value is a date
    if pd.to_datetime(cell_value, errors='coerce') is not pd.NaT:
        return cell_value
    else:
        return str(cell_value)


def clean_txt(cell_value):
    if cell_value.startswith("['") and cell_value.endswith("']"):
        return cell_value[2:-2]
    else:
        return cell_value

def transform_txtrawr(cell_value):
    match = re.match(r'^\d+(\.\d+)?\s(.*)', cell_value)
    if match:
        return match.group(2)
    else:
        return cell_value
    

df_original = df.copy()

df[['question', 'model_answer', 'student_answer']] = df[['question', 'model_answer', 'student_answer']].apply(transform_text)
df[['question', 'model_answer', 'student_answer']] = df[['question', 'model_answer', 'student_answer']].apply(lambda x: x.lower() if isinstance(x, str) else x)
mask = df['source'] == 'short_answer_grading.csv'
df.loc[mask, ['question', 'model_answer', 'student_answer']] = df.loc[mask, ['question', 'model_answer', 'student_answer']].applymap(transform_txtrawr)
#df.loc[df['source'] == 'duorc.csv', ['model_answer']] = df.loc[df['source'] == 'duorc.csv', ['model_answer']].apply(lambda x: x.str.replace(r'^[\W_]+$', '', regex=True))
#df.loc[df['source'] == 'duorc.csv', ['model_answer', 'student_answer']] = df.loc[df['source'] == 'duorc.csv', ['model_answer', 'student_answer']].apply(lambda x: x.apply(clean_txt))
df = df[(df['student_answer'] != "no answer present.")]
df["student_answer"] = df["student_answer"].str.replace('<stop>', '')
df.drop(df["student_answer"][df["student_answer"].isnull()].index, inplace=True)
df.drop(df["model_answer"][df["model_answer"].isnull()].index, inplace=True)
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# print(df[df['student_answer'] == 'no answer present.'])
# print(df[df['source'] == 'short_answer_grading.csv'])
# print(df[df['source'] == 'duorc.csv'])
print(df)

# Compare the original and modified dataframes
# changed_rows = (df != df_original).any(axis=1)
# changed_count = changed_rows.sum()
# print(f"Number of rows affected and changed: {changed_count}")

# df.head()
df.to_csv("final_version.csv")
# filtered_data = df[df['model_answer'].str.startswith("['") & df['model_answer'].str.endswith("']")]
# print(filtered_data)
# filtered_data = df[df['question'].str.match(r'^\d')]
# print(filtered_data)

In [ ]:
df.drop_duplicates(subset=['question','student_answer', 'model_answer'], inplace=True)
df.to_csv("ffinal_version.csv")

In [ ]:
df = pd.read_csv(r"E:\Y4\Grad Proj\AGS\ffinal_version.csv")
dff = pd.read_csv(r"E:\Y4\Grad Proj\AGS\duorc_corrected.csv")
dff["source"] = ["duorc.csv"]*len(dff) 
# Concatenate the two DataFrames
concatenated_df = pd.concat([df, dff], ignore_index=True)
# concatenated_df.drop(['Unnamed: 0', 'Unnamed: 0.2', 'Unnamed: 0.1'], axis=1, inplace=True)
# concatenated_df["source"] = concatenated_df["source"].apply(lambda x: x if isinstance(x, str) else "duorc.csv")
concatenated_df.to_csv("ffinal_version.csv")

In [ ]:
df = pd.read_csv(r"E:\Y4\Grad Proj\AGS\all_corrected.csv")

# Store the original dataframe for comparison
df_original = df.copy()

def print_changes(step_description, changed_rows):
    changed_count = changed_rows.sum()
    print(f"After '{step_description}', number of rows affected and changed: {changed_count}")

# Step 1: Apply clean_txt to 'model_answer' and 'student_answer' columns for rows with source 'duorc.csv'
mask_step1 = df['source'] == 'duorc.csv'
df_original_step1 = df_original[mask_step1].copy()
df.loc[mask_step1, ['model_answer', 'student_answer']] = df.loc[mask_step1, ['model_answer', 'student_answer']].apply(lambda x: x.apply(clean_txt))
print_changes("clean_txt for 'model_answer' and 'student_answer' in duorc.csv", (df[mask_step1] != df_original_step1).any(axis=1))

# Step 2: Apply transform_text to 'question', 'model_answer', and 'student_answer' columns
df_original_step2 = df_original.copy()
df[['question', 'model_answer', 'student_answer']] = df[['question', 'model_answer', 'student_answer']].apply(transform_text)
print_changes("transform_text for 'question', 'model_answer', and 'student_answer'", (df != df_original_step2).any(axis=1))

# Step 3: Lowercase 'question', 'model_answer', and 'student_answer' columns
df_original_step3 = df_original.copy()
df[['question', 'model_answer', 'student_answer']] = df[['question', 'model_answer', 'student_answer']].apply(lambda x: x.lower() if isinstance(x, str) else x)
print_changes("lowercase 'question', 'model_answer', and 'student_answer'", (df != df_original_step3).any(axis=1))

# Step 4: Apply transform_txtrawr to 'question', 'model_answer', and 'student_answer' columns for rows with source 'short_answer_grading.csv'
mask_step4 = df['source'] == 'short_answer_grading.csv'
df_original_step4 = df_original[mask_step4].copy()
df.loc[mask_step4, ['question', 'model_answer', 'student_answer']] = df.loc[mask_step4, ['question', 'model_answer', 'student_answer']].applymap(transform_txtrawr)
print_changes("transform_txtrawr for 'question', 'model_answer', and 'student_answer' in short_answer_grading.csv", (df[mask_step4] != df_original_step4).any(axis=1))

# Step 5: Apply regex replacement to 'model_answer' column for rows with source 'duorc.csv'
mask_step5 = df['source'] == 'duorc.csv'
df_original_step5 = df_original[mask_step5].copy()
df.loc[mask_step5, ['model_answer']] = df.loc[mask_step5, ['model_answer']].apply(lambda x: x.str.replace(r'^[\W_]+$', '', regex=True))
print_changes("regex replacement for 'model_answer' in duorc.csv", (df[mask_step5] != df_original_step5).any(axis=1))